# 03 - Feature EngineeringCreate weekly aggregations and lag features for forecasting.

## Load Clean Data

In [ ]:
import pandas as pdimport numpy as npfrom pathlib import Pathimport warningswarnings.filterwarnings('ignore')BASE_PATH = Path('..')FEATURES_PATH = BASE_PATH / 'features_v2'df = pd.read_csv(FEATURES_PATH / 'transactions_clean.csv', low_memory=False)print(f"Loaded: {len(df):,} rows")# Parse datesif 'order_date' in df.columns:    df['order_date'] = pd.to_datetime(df['order_date'])    print(f"Date range: {df['order_date'].min()} to {df['order_date'].max()}")

## Create Year-Week Column

In [ ]:
# Create year_week if not existsif 'year_week' not in df.columns:    df['year_week'] = df['order_date'].dt.strftime('%G-W%V')print(f"Unique weeks: {df['year_week'].nunique()}")print(f"Sample weeks: {sorted(df['year_week'].unique())[:10]}")# Extract week numberdf['week_num'] = df['year_week'].str.extract(r'W(\d+)').astype(int)

## SKU Weekly Aggregation

In [ ]:
# Aggregate by SKU and weeksku_weekly = df.groupby(['sku', 'year_week', 'week_num']).agg({    'quantity': 'sum',    'line_total': 'sum',    'invoice_id': 'nunique',    'customer_id': 'nunique' if 'customer_id' in df.columns else 'count'}).reset_index()sku_weekly.columns = ['sku', 'year_week', 'week_num', 'weekly_quantity', 'weekly_revenue', 'num_orders', 'num_customers']print(f"SKU weekly aggregation: {len(sku_weekly):,} rows")print(f"Unique SKUs: {sku_weekly['sku'].nunique():,}")sku_weekly.head()

## Create Lag Features

In [ ]:
# Sort by SKU and weeksku_weekly = sku_weekly.sort_values(['sku', 'week_num'])# Create lag featuresfor lag in [1, 2, 4]:    sku_weekly[f'lag{lag}_quantity'] = sku_weekly.groupby('sku')['weekly_quantity'].shift(lag)    sku_weekly[f'lag{lag}_revenue'] = sku_weekly.groupby('sku')['weekly_revenue'].shift(lag)# Rolling averagessku_weekly['rolling_avg_4w'] = sku_weekly.groupby('sku')['weekly_quantity'].transform(    lambda x: x.shift(1).rolling(4, min_periods=1).mean())sku_weekly['rolling_avg_4w_revenue'] = sku_weekly.groupby('sku')['weekly_revenue'].transform(    lambda x: x.shift(1).rolling(4, min_periods=1).mean())print("Lag features created:")print(sku_weekly.columns.tolist())

## Add Time Features

In [ ]:
# Week of year featuressku_weekly['is_month_start'] = sku_weekly['week_num'].isin([1, 5, 9, 14, 18, 22, 27, 31, 36, 40, 44, 48])sku_weekly['is_month_end'] = sku_weekly['week_num'].isin([4, 8, 13, 17, 21, 26, 30, 35, 39, 43, 47, 52])sku_weekly['is_w47'] = sku_weekly['week_num'] == 47  # Black Fridaysku_weekly['quarter'] = ((sku_weekly['week_num'] - 1) // 13) + 1print("Time features added")sku_weekly[['year_week', 'week_num', 'is_w47', 'quarter']].head(10)

## Create H1/H2 Split

In [ ]:
# H1 = Weeks 1-26 (training), H2 = Weeks 27-52 (testing)sku_weekly['period'] = sku_weekly['week_num'].apply(lambda w: 'H1' if w <= 26 else 'H2')# Add date column (first day of week)sku_weekly['date'] = pd.to_datetime(sku_weekly['year_week'] + '-1', format='%G-W%V-%u')print(f"H1 (training): {(sku_weekly['period'] == 'H1').sum():,} rows")print(f"H2 (testing): {(sku_weekly['period'] == 'H2').sum():,} rows")

## Save Feature Tables

In [ ]:
# Save full weekly featuressku_weekly.to_csv(FEATURES_PATH / 'v2_features_weekly.csv', index=False)print(f"✓ Saved v2_features_weekly.csv: {len(sku_weekly):,} rows")# Save H1/H2 splits for forecastingh1 = sku_weekly[sku_weekly['period'] == 'H1']h2 = sku_weekly[sku_weekly['period'] == 'H2']h1.to_csv(FEATURES_PATH / 'forecast_sku_weekly_H1.csv', index=False)h2.to_csv(FEATURES_PATH / 'forecast_sku_weekly_H2.csv', index=False)print(f"✓ Saved forecast_sku_weekly_H1.csv: {len(h1):,} rows")print(f"✓ Saved forecast_sku_weekly_H2.csv: {len(h2):,} rows")

## Category Aggregation

In [ ]:
# Load product info for categoryif (FEATURES_PATH / 'v2_dim_products.csv').exists():    products = pd.read_csv(FEATURES_PATH / 'v2_dim_products.csv')    sku_to_cat = dict(zip(products['sku'].astype(str), products['category_l1']))        # Add category to weekly data    sku_weekly['category'] = sku_weekly['sku'].astype(str).map(sku_to_cat).fillna('Unknown')        # Aggregate by category    cat_weekly = sku_weekly.groupby(['category', 'year_week', 'week_num', 'period']).agg({        'weekly_quantity': 'sum',        'weekly_revenue': 'sum',        'sku': 'nunique'    }).reset_index()        cat_weekly.columns = ['category', 'year_week', 'week_num', 'period', 'weekly_quantity', 'weekly_revenue', 'num_skus']        cat_weekly.to_csv(FEATURES_PATH / 'v2_features_category.csv', index=False)    print(f"✓ Saved v2_features_category.csv: {len(cat_weekly):,} rows")else:    print("No product dimension file found, skipping category aggregation")